# The 8 schools example

The "8 schools" example is a famous example of a hierarchical (multi-level) model with so-called "partial pooling." The data comes from a study of coaching programs for the Verbal SAT (for those unfamiliar, the SAT is one of two major standardized tests used in undergraduate college/university admissions in the US).

Let's do our imports and load the data.

In [ ]:
import numpy as np, scipy as sp, pymc3 as pm, pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

d = {'School': ['A','B','C','D','E','F','G','H'],
     'Effect': [28, 8, -3, 7, -1, 1, 18, 12],
     'SE': [15, 10, 16, 11, 9, 11, 10, 18]}
schools = pd.DataFrame(data=d)
schools

The estimates $y_i$ of the effects are obtained by independent experiments carried out at each school. The estimates are not simple sample means, as they underwent some adjustment procedures, but they can be assumed to have approximately normal sampling distributions with known standard errors (in the above table).

## Hierarchical model


### The model

We presume that the estimates $y_i$ are produced by 8 independent normal distributions with their own means $\theta_i$ and variances $\sigma^2_i$:

$$ \begin{align}
y_i | \theta_i, \sigma^2_i &\sim \mathrm{Normal}(\theta_i, \sigma^2_i) \\
\theta_i &\sim \mathrm{Normal}(\mu, \tau)\\
\mu &\sim \mathrm{Normal}(0,5) \\
\tau &\sim \mathrm{HalfCauchy}(5) 
\end{align} \\$$

We presume that the parameters $\theta_i$ are themselves drawn from a normal distribution with parameters $\mu, \tau$. Given $\mu, \tau$, we assume the $\theta_i$'s to be conditionally independent.

For convenience, we presume the sampling standard deviations of the $\bar y_i$ to be known and equal to the values in the data table. 

In [ ]:
with pm.Model() as centered_model:
    # Hyperparameters
    tau = pm.HalfCauchy('tau', 5)
    mu = pm.Normal('mu', mu = 0, sigma = 5)
    
    # Parameter vector   
    theta = pm.Normal('theta', mu = mu, sigma = tau, shape = 8)
    
    # Data likelihood
    y_obs = pm.Normal('y_obs', mu = theta, sigma = schools['SE'], observed = schools['Effect'])
    
    trace = pm.sample(5000, target_accept = 0.9, chains = 4, cores = 4, tune = 1000)

In [ ]:
pm.summary(trace)

In [ ]:
def pairplot_divergence(trace, ax=None, divergence=True, color='C3', divergence_color='C2'):
    theta = trace.get_values(varname='theta', combine=True)[:, 0]
    logtau = trace.get_values(varname='tau_log__', combine=True)
    if not ax:
        _, ax = plt.subplots(1, 1, figsize=(10, 5))
    ax.plot(theta, logtau, 'o', color=color, alpha=.5)
    if divergence:
        divergent = trace['diverging']
        ax.plot(theta[divergent], logtau[divergent], 'o', color=divergence_color)
    ax.set_xlabel('theta[0]')
    ax.set_ylabel('log(tau)')
    ax.set_title('scatter plot between log(tau) and theta[0]');
    return ax

pairplot_divergence(trace)

In [ ]:
K = 8

mu = 5
tau = 3.5
sigma = 12
theta_true = sp.stats.norm.rvs(mu, tau, K)
y = sp.stats.norm.rvs(theta_true, sigma)
y

In [ ]:
theta_true

In [ ]:
with pm.Model() as centered_model:
    # Hyperparameters
    mu = pm.Normal('mu', mu = 0, sigma = 5)
    #mu = pm.Flat('mu')
    tau = pm.HalfCauchy('tau', 5)
    
    # Parameter vector
    theta = pm.Normal('theta', mu = mu, sigma = tau, shape = K)
    
    # Data likelihood
    y_obs = pm.Normal('y_obs', mu = theta, sigma = sigma, observed = y)
    
    trace = pm.sample(4000, target_accept = 0.90, chains = 4, cores = 4, tune = 1000)

In [ ]:
pm.summary(trace, var_names = ['mu', 'tau'])

In [ ]:
with pm.Model() as noncentered_model:
    # Hyperparameters
    mu = pm.Normal('mu', mu = 0, sigma = 5)
    tau = pm.HalfCauchy('tau', 5)
    eta = pm.Normal('eta', mu = 0, sigma = 1, shape = K)
    
    # Parameter vector
    theta = pm.Deterministic('theta', mu + tau * eta)
    
    # Data likelihood
    y_obs = pm.Normal('y_obs', mu = theta, sigma = sigma, observed = y)
    
    trace = pm.sample(4000, target_accept = 0.90, chains = 4, cores = 4, tune = 1000)

In [ ]:
pm.summary(trace, var_names = ['mu', 'tau'])

In [ ]:
with pm.Model() as noncentered_model:
    # Hyperparameters
    tau = pm.HalfCauchy('tau', 5)
    mu = pm.Normal('mu', mu = 0, sigma = 5)
    eta = pm.Normal('eta', 0, 1, shape = 8)
    
    # Parameter vector
    
    theta = pm.Deterministic('theta', mu + tau * eta)
    #theta = pm.Normal('theta', mu = mu, sigma = tau, shape = 8)
    
    # Data likelihood
    y_obs = pm.Normal('y_obs', mu = theta, sigma = schools['SE'], observed = schools['Effect'])
    
    trace = pm.sample(5000, target_accept = 0.95, chains = 4, cores = 4, tune = 1000)

In [ ]:
pm.summary(trace, var_names=['mu', 'theta', 'tau'])

In [ ]:
pm.forestplot(trace, var_names = ['theta'])